In [2]:
import os
import json
from matplotlib import pyplot as plt
import torch
from task import Task, Blackboard
from tqdm import tqdm
import trainer


In [3]:
trainer.precompute_tasks(
    input_dir="data/evaluation", 
    output_dir="precomputed_tasks/evaluation_400",
    augmentation_types=None
)

Loaded 0 original tasks from data/evaluation
Precomputing complete: 0 original tasks and 0 augmented tasks saved to precomputed_tasks/evaluation_400


In [4]:
task = trainer.load_precomputed_task("precomputed_tasks/007bbfb7_rotate_90.pt")
print(task.task_id, len(task.train_graphs), len(task.test_graphs))

FileNotFoundError: [Errno 2] No such file or directory: 'precomputed_tasks/007bbfb7_rotate_90.pt'